In [3]:
import pandas as pd
import glob


In [4]:
path=r'/Users/tomas/C Finance Oy/Analysis - Documents/Dashboard/openlyUnemployedSE/'

In [5]:
import re
all_files = glob.glob(path + "/*.csv")

li = []
cols=['Vecka', '2020']
df_init=pd.DataFrame(data=range(53), columns=['Vecka'])+1
for filename in all_files:
    if  not re.search('name_mapping', filename): 
        lan_name=filename.split('/')[7].split('.csv')[0]
        df = pd.read_csv(filename, index_col=None, header=0, delimiter=';')[cols].dropna()
        df.columns=['Vecka', lan_name]
        df.Vecka = df.Vecka.astype(str).astype(int)
        df_init=df_init.merge(df, left_on='Vecka', right_on='Vecka', how='left')

    #    li.append(df)
df_init.dropna(inplace=True)
df_init = df_init.set_index('Vecka')
#frame = pd.concat(li, axis=0, ignore_index=True)

In [6]:
df_init.columns

Index(['dalarna', 'orebro', 'uppsala', 'varmland', 'halland', 'kronoberg',
       'vastragotaland', 'gavleborg', 'skane', 'stockholm', 'jamtland',
       'vastmanland', 'ostergotland', 'blekinge', 'kalmar', 'norrbotten',
       'gotland', 'vasternorrland', 'sodermanland', 'jonkoping',
       'vasterbotten'],
      dtype='object')

In [6]:
# The reference week for unemployment is w 13
ref_week=13
df_rel=(df_init/df_init.loc[ref_week])
df_rel.head(5)

,dalarna,orebro,uppsala,varmland,halland,kronoberg,vastragotaland,gavleborg,skane,stockholm,...,vastmanland,ostergotland,blekinge,kalmar,norrbotten,gotland,vasternorrland,sodermanland,jonkoping,vasterbotten
Vecka,,,,,,,,,,,,,,,,,,,,,
1,1.004685,1.005228,0.963019,1.009787,1.037332,0.992886,0.995731,1.063774,1.039866,0.952839,...,1.007174,1.070217,1.122124,1.045717,1.055363,0.986628,1.061078,1.027178,1.000662,1.021702
2,1.005060,1.007288,0.967869,1.008606,1.027221,0.994528,0.991310,1.053499,1.034258,0.950723,...,1.008768,1.065418,1.084956,1.056770,1.037321,0.978988,1.045137,1.027588,1.002317,1.022667
3,1.001687,1.011724,0.972113,1.006581,1.031110,1.000274,0.992682,1.046811,1.030637,0.950037,...,1.002072,1.060971,1.085546,1.066566,1.037815,0.979943,1.044421,1.023218,0.999504,1.025561
4,1.002624,1.004911,0.972871,1.001350,1.019638,1.003010,0.986279,1.045180,1.022488,0.945157,...,0.990116,1.043768,1.062242,1.056016,1.025210,0.966571,1.031883,1.016116,0.996194,1.020497
5,0.993816,0.998574,0.948924,1.000000,1.006416,1.003557,0.976064,1.031969,1.018604,0.944967,...,0.978957,1.037917,1.052802,1.046973,1.013099,0.974212,1.028121,1.006009,0.991726,1.018809


In [15]:
# reformat to
# Lan, Vecka, relative_unemployment
df_final=df_rel.unstack(0).reset_index()
df_final.columns=['Lan', 'Vecka', 'relative_unemployment']
#df_final.to_pickle('../data/all_weeks_counties.pkl')
df_final.head(5)


,Lan,Vecka,relative_unemployment
0,dalarna,1,1.004685
1,dalarna,2,1.005060
2,dalarna,3,1.001687
3,dalarna,4,1.002624
4,dalarna,5,0.993816


In [18]:
name_id=(pd.read_csv("/Users/tomas/C Finance Oy/Analysis - Documents/Dashboard/openlyUnemployedSE/name_mapping.csv", 
             delimiter=';')[['ID','LAN_SHORTNAME','LAN']]
)
(df_final.
 merge(name_id, left_on='Lan', right_on='LAN_SHORTNAME', how='inner').
 to_pickle('../data/all_weeks_counties.pkl')
)